# Baseline Pack: Testing Direction Specificity

This notebook runs minimal baselines to validate that projection ablation effects are direction-specific, not artifacts of ablating any random direction.

**Baselines tested:**
1. **Random directions** (n=10) - Tests if any direction works
2. **Shuffled contrast** - Tests if per-prompt pairing matters
3. **Benign contrast** - Tests domain specificity

**Expected runtime:** ~20-30 minutes on T4 GPU, ~15-20 minutes on A100

---

## Setup

### 1. Clone Repository

In [ ]:
!git clone https://github.com/isahan78/steering-reliability.git
%cd steering-reliability
!pwd

### 2. Install Dependencies

In [ ]:
# Uninstall any conflicting packages
!pip uninstall -y numpy pandas datasets transformer-lens transformers pyarrow scikit-learn -q

# Install all dependencies in one command (ensures compatibility)
!pip install --no-cache-dir numpy pandas torch transformer-lens transformers datasets matplotlib seaborn pyyaml tqdm pyarrow scikit-learn

# Add src to Python path
import sys
sys.path.insert(0, '/content/steering-reliability/src')

# Check GPU
import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

### 3. Verify Imports (CRITICAL)

In [ ]:
import sys
sys.path.insert(0, '/content/steering-reliability/src')

print("="*60)
print("IMPORT VERIFICATION")
print("="*60)

try:
    import numpy as np
    print(f"✓ numpy {np.__version__}")
    
    from transformer_lens import HookedTransformer
    print(f"✓ transformer_lens.HookedTransformer")
    
    from steering_reliability.config import load_config
    print("✓ config module")
    
    from steering_reliability.model import load_model
    print("✓ model module")
    
    from steering_reliability.directions.random_direction import sample_random_direction
    print("✓ random_direction module")
    
    from steering_reliability.directions.build_direction import build_shuffled_contrast_direction
    print("✓ shuffled_contrast module")
    
    from steering_reliability.directions.benign_contrast import build_benign_contrast_direction
    print("✓ benign_contrast module")
    
    print("\n" + "="*60)
    print("✅ SUCCESS! All imports work.")
    print("="*60)
    print("\nReady to run baseline pack!")
    
except Exception as e:
    print("\n" + "="*60)
    print("❌ FAILED!")
    print("="*60)
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

---

## Run Baseline Pack

This will:
1. Load GPT-2 Small
2. Build learned direction (from harm_train)
3. Build shuffled contrast direction
4. Generate 10 random directions
5. Build benign contrast direction
6. Test all directions with ablation

**Settings:**
- Layer: 10
- Alphas: {0, 1, 4, 8}
- Prompts: 50 harm_test, 50 benign
- Random trials: 10

In [ ]:
%%time
# Run baseline pack experiment
!python scripts/run_baseline_pack.py \
  --layer 10 \
  --alphas 0 1 4 8 \
  --n_harm_test 50 \
  --n_benign 50 \
  --n_random 10 \
  --seed 0 \
  --include_benign_contrast \
  --output_dir artifacts/baselines

---

## Generate Figures

Create visualizations from the baseline pack results.

In [ ]:
!python -m steering_reliability.analysis.plot_baseline_pack \
  --in_parquet artifacts/baselines/baseline_pack_results.parquet \
  --out_dir artifacts/baselines/figures

---

## Analyze Results

### View Summary Table

In [ ]:
import pandas as pd

# Read summary table
table = pd.read_csv('artifacts/baselines/figures/baseline_pack_table.csv')

print("="*80)
print("BASELINE PACK SUMMARY TABLE")
print("="*80)
print(table.to_string(index=False))
print()

# Highlight key findings
print("="*80)
print("KEY FINDINGS")
print("="*80)

# Get learned and random at alpha=8
learned_8 = table[(table['direction_type'] == 'learned')]
random_8 = table[(table['direction_type'] == 'random')]

if len(learned_8) > 0:
    learned_refusal = learned_8['harm_refusal_α8.0'].values[0]
    print(f"\n✓ Learned direction (α=8): {learned_refusal} refusal on harm_test")

if len(random_8) > 0:
    random_refusal = random_8['harm_refusal_α8.0'].values[0]
    print(f"✓ Random baseline (α=8): {random_refusal} refusal on harm_test")
    print(f"\n→ Learned direction is direction-specific!" if learned_8['harm_refusal_α8.0'].values[0] > random_8['harm_refusal_α8.0'].values[0] else "→ WARNING: Random matches learned!")

### View Direction Specificity Figure

In [ ]:
from IPython.display import Image, display

print("="*80)
print("DIRECTION SPECIFICITY: Refusal on Harm Test")
print("="*80)
print("\nLearned (green) should be >> Random (gray) and Shuffled (blue)\n")

display(Image(filename='artifacts/baselines/figures/direction_specificity.png'))

### View Benign Preservation Figure

In [ ]:
print("="*80)
print("BENIGN PRESERVATION: Helpfulness on Benign Queries")
print("="*80)
print("\nAll directions should maintain high helpfulness (>95%)\n")

display(Image(filename='artifacts/baselines/figures/benign_preservation.png'))

---

## Detailed Analysis

Load full results and compute statistics.

In [ ]:
import pandas as pd
import numpy as np

# Load full results
df = pd.read_parquet('artifacts/baselines/baseline_pack_results.parquet')

print("="*80)
print("DETAILED ANALYSIS (α=8 on harm_test)")
print("="*80)

# Filter to alpha=8, harm_test
harm_8 = df[(df['alpha'] == 8) & (df['split'] == 'harm_test')]

# Compute statistics by direction type
stats = harm_8.groupby('direction_type').agg({
    'is_refusal': ['mean', 'std', 'count']
}).round(3)

print("\nRefusal rates by direction type:")
print(stats)

# For random, show distribution across trials
print("\n" + "="*80)
print("RANDOM DIRECTION TRIALS (α=8, harm_test)")
print("="*80)

random_trials = harm_8[harm_8['direction_type'] == 'random'].groupby('random_trial').agg({
    'is_refusal': 'mean'
}).round(3)

print(random_trials)
print(f"\nRandom trials mean: {random_trials['is_refusal'].mean():.3f}")
print(f"Random trials std: {random_trials['is_refusal'].std():.3f}")

# Compare learned vs random
learned_refusal = harm_8[harm_8['direction_type'] == 'learned']['is_refusal'].mean()
random_refusal = harm_8[harm_8['direction_type'] == 'random']['is_refusal'].mean()
gap = learned_refusal - random_refusal

print("\n" + "="*80)
print("LEARNED vs RANDOM GAP")
print("="*80)
print(f"Learned: {learned_refusal:.1%}")
print(f"Random:  {random_refusal:.1%}")
print(f"Gap:     {gap:+.1%}")

if gap > 0.3:
    print("\n✅ PASS: Large gap indicates direction is specific!")
elif gap > 0.1:
    print("\n⚠️  MODERATE: Gap exists but could be larger")
else:
    print("\n❌ FAIL: Random matches learned - effect may not be direction-specific!")

---

## Download Results

Download all baseline pack results as a ZIP file.

In [ ]:
# Create ZIP of all baseline results
!zip -r baseline_pack_results.zip artifacts/baselines/

# Download
from google.colab import files
files.download('baseline_pack_results.zip')

print("\n✓ Results downloaded!")
print("\nExtract this ZIP and use the following files for your MATS application:")
print("  - artifacts/baselines/figures/direction_specificity.png (main figure)")
print("  - artifacts/baselines/figures/baseline_pack_table.csv (summary table)")
print("  - artifacts/baselines/baseline_pack_results.parquet (full data)")

---

## Interpretation Guide

### What You Should See (If Claim is Valid)

**1. Direction Specificity (harm_test, α=8)**
- ✅ Learned: ~95-98% refusal
- ✅ Random (mean): <50% refusal
- ✅ Shuffled: Weaker than learned (shows pairing matters)
- ✅ Benign contrast: Low refusal (shows domain specificity)

**2. Benign Preservation**
- ✅ All directions: >95% helpfulness
- ✅ No catastrophic degradation

**3. Gap Interpretation**
- Gap >30%: **Strong evidence** direction is specific
- Gap 10-30%: **Moderate evidence** (acceptable)
- Gap <10%: **Weak/no evidence** (concerning)

### Adding to Your Application

**Text (4-6 sentences):**

> To test whether the ablation effect is direction-specific, we implemented three baselines: (1) random direction ablation (n=10 trials, sampled from N(0,I)) tests whether removing any projection disrupts compliance, (2) shuffled contrast tests whether per-prompt alignment matters by breaking the refusal/compliance pairing, and (3) benign contrast tests whether contrast directions generically induce refusal. The learned direction achieves [XX]% refusal on harm_test at α=8, compared to random mean of [YY]±[ZZ]% and shuffled [WW]%, while maintaining [PP]% benign helpfulness across all conditions. This [large/moderate] gap rules out the hypothesis that ablation is a generic "compliance suppressor" and supports the claim that the learned direction captures a specific harmful-compliance feature.

**Figures:**
- Insert `direction_specificity.png` in Results section
- Insert `baseline_pack_table.csv` as table in Results section

---

## Summary

**What was tested:**
- Learned direction (contrastive, aligned)
- Shuffled direction (contrastive, misaligned)
- Random directions (10 trials)
- Benign contrast direction

**Key question answered:**
Is the 98% refusal rate from ablation due to the specific learned direction, or would any direction work?

**Expected result:**
Learned >> Random (large gap) → Direction is specific → Claim is valid!

---